
<h1 align="center">Cell Tracking according to Biological Needs</h1>
<h2 align="center">Strong Mitosis-aware Random-finite Sets Tracker with Aleatoric Uncertainty</h2>

<p align="center">
  <img alt="Teaser Image" src="./assets/teaser.png">
  <br>
    <em>The output distribution of EmbedTrack using different input transformations on BF-C2DL-HSC.</em>
</p>

# Select the Dataset you want to process

In [1]:
# Select the dataset you want to process
DATASET = 'DIC-C2DH-HeLa'
SUBSET = 'train'
SEQUENCE = '01'

# Select some Inference Parameter
SHIFTS = 50
MULTISCALE = False
MULTI_SEGMENTATION = False

# Check if the selection is valid
AVAILABLE_DATASETS = [
    "Fluo-N2DH-SIM+",
    "Fluo-C2DL-MSC",
    "Fluo-N2DH-GOWT1",
    "PhC-C2DL-PSC",
    "BF-C2DL-HSC",
    "Fluo-N2DL-HeLa",
    "BF-C2DL-MuSC",
    "DIC-C2DH-HeLa",
    "PhC-C2DH-U373",
]
AVAILABLE_SUBSETS = ['train', 'challenge']
AVAILABLE_SEQUENCES = ['01', '02']
assert DATASET in AVAILABLE_DATASETS, f"Invalid dataset: {DATASET}"
assert SUBSET in AVAILABLE_SUBSETS, f"Invalid subset: {SUBSET}"
assert SEQUENCE in AVAILABLE_SEQUENCES, f"Invalid sequence: {SEQUENCE}"

# Setting up the environment

Creating the environment
  - downloading the CTC data
  - downloading the pretrained models. 

In [2]:
# Download the data
if SUBSET == 'train':
    !python utils/prepare_data.py --datasets $DATASET --train
else:
    !python utils/prepare_data.py --datasets $DATASET --challenge

In [3]:
# Download the pretrained models
import zipfile

!wget https://www.tnt.uni-hannover.de/de/project/MPT/data/BiologicalNeeds/models.zip
with zipfile.ZipFile("models.zip") as z:
    z.extractall("")

--2024-05-08 17:31:30--  https://www.tnt.uni-hannover.de/de/project/MPT/data/BiologicalNeeds/models.zip
Resolving www.tnt.uni-hannover.de (www.tnt.uni-hannover.de)... 130.75.31.71
Connecting to www.tnt.uni-hannover.de (www.tnt.uni-hannover.de)|130.75.31.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248308697 (237M) [application/zip]
Saving to: ‘models.zip.3’

models.zip.3        100%[===================>] 236.80M   112MB/s    in 2.1s    

2024-05-08 17:31:32 (112 MB/s) - ‘models.zip.3’ saved [248308697/248308697]



# Inference

## 1) EmbedTrack with Aleatoric Uncertainty by Advanced Test-Time Augmentation

In [4]:
import os
from embedtrack.scripts.inference import process

res_path = os.path.join(os.getcwd(), 'results/embedtrack')
process(
    train=True if SUBSET == 'train' else False,
    challenge=True if SUBSET == 'challenge' else False,
    dataset=DATASET,
    sequence=SEQUENCE,
    res_path=res_path,
    shifts=SHIFTS,
    multiscale=MULTISCALE,
    multi_segmentation=MULTI_SEGMENTATION,
)

/home/kaiser/code_dev
['/home/kaiser/code_dev/BiologicalNeeds/Data/train']
Shifts: [0, 50]
Multiscale: False
Multisegmentation: False
`model_dict` dictionary successfully created with: 
 -- num of classes equal to [4, 1, 2], 
 -- input channels equal to 1, 
 -- name equal to 2d
Creating branched erfnet with [4, 1, 2] classes
Save tracking mask t083.tif
Save tracking mask t082.tif
Save tracking mask t081.tif
Save tracking mask t080.tif
Save tracking mask t079.tif
Save tracking mask t078.tif
Save tracking mask t077.tif
Save tracking mask t076.tif
Save tracking mask t075.tif
Save tracking mask t074.tif
Save tracking mask t073.tif
Save tracking mask t072.tif
Save tracking mask t071.tif
Save tracking mask t070.tif
Save tracking mask t069.tif
Save tracking mask t068.tif
Save tracking mask t067.tif
Save tracking mask t066.tif
Save tracking mask t065.tif
Save tracking mask t064.tif
Save tracking mask t063.tif
Save tracking mask t062.tif
Save tracking mask t061.tif
Save tracking mask t060.tif
S

## 2) Multi Hypothesis Tracking

In [6]:
from mht.scripts.inference import infer_all

in_path = os.path.join(os.getcwd(), 'results/embedtrack')
res_path = os.path.join(os.getcwd(), 'results/mht')

infer_all(
    data_root=in_path,
    dest_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
    tracker_args={},
)

Infer train DIC-C2DH-HeLa 01
PMBM tracker parameters:
    Debug mode: False
    Multiprocessing: True
    Mitosis extension: True
    mitosis_min_length_a0: 80
    max_number_of_hypotheses: 250
    max_sampling_hypotheses: 3
    gating_probability: 0.01
    gating_distance: 10
    min_sampling_increment: 0.01
    min_object_probability: 0.1
    use_kalman_filter: False
    split_likelihood: 0.25
    P_S: 0.9
    P_B: 0.1
    P_B_border: 0.35
    system_uncertainty: 0.0
    segmentation_errors: True
(After Processing Frame 0) PMBMTracker(hypotheses=1, top_likelihood=1.0, branch_sw=0, processing_time=0:00:00, )
(After Processing Frame 1) PMBMTracker(hypotheses=2, top_likelihood=0.9482545925622464, branch_sw=0, processing_time=0:00:00.016925, )
(After Processing Frame 2) PMBMTracker(hypotheses=3, top_likelihood=0.9117965591039993, branch_sw=0, processing_time=0:00:00.056212, )
(After Processing Frame 3) PMBMTracker(hypotheses=3, top_likelihood=0.9357051313536623, branch_sw=0, processing_t

100%|██████████| 84/84 [00:00<00:00, 1673.57it/s]


## 3) Interpolation to fill temporal gaps in trajectories

In [7]:
from utils.interpolate import postprocess_all

in_path = os.path.join(os.getcwd(), 'results/mht')
res_path = os.path.join(os.getcwd(), 'results/interpolated')

postprocess_all(
    data_root=in_path,
    dest_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
)

Interpolate train DIC-C2DH-HeLa 01
    Add mask at frame 39 with id 17
    Add mask at frame 68 with id 21


## 4) Postprocessing to satisfy the CTC requirements

In [8]:
from utils.postprocess import postprocess_all

res_path = os.path.join(os.getcwd(), 'results/interpolated')

postprocess_all(
    data_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
)

Process train DIC-C2DH-HeLa 01
